In [9]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
import datetime
import os
from concurrent.futures import ThreadPoolExecutor

writeToFile = True

#This is the current graph file index
currentGraphFile = 0

#This is how many G6 strings go into each file
graphFileSize = 500

foundGraphs = 0

#This is what is put into the next file that we save
outputLine = ""

#This is the current output stream
outputStream = None
    
# For a memory cost, we can use this to avoid doing the checks twice for subgraphs when we remove nodes
criticalCache = {}

# This is to test the equality between the Graph6 keys and the str() function keys
testStrCache = {}

#We can also cache the Graph6 strings
graph6Cache = {}

#This is where the g6 files will be stored
imgDirectory = "critical graphs"

def has_valid_coloring(adjacency_dict):

    # graph6DictString = graph_to_graph6_3(adjacency_dict)
    STRdictString = str(adjacency_dict)
    
    # getter = criticalCache.get(graph6DictString, None)
    getterStr = testStrCache.get(STRdictString, None)
    
    if (getterStr != None):
            # print(criticalCache[graph6DictString] == testStrCache[STRdictString])
            global testInteger
            testInteger = testInteger + 1
            return getterStr

    # Determine the maximum degree of any node in the graph
    node_degrees = [len(neighbors) for neighbors in adjacency_dict.values()]
    
    if (len(node_degrees) == 0):
        cache[dictString] = 0
        return 0
    
    max_degree = max(node_degrees)

    # Define a function that recursively tries all possible colorings
    def try_coloring(coloring, node_order):
        # If all nodes have been colored, check if the coloring is valid
        if len(coloring) == len(adjacency_dict):
            for node, neighbors in adjacency_dict.items():
                for neighbor in neighbors:
                    
                    # Normally this check isn't needed, but we need it when we remove nodes from the dictionary
                    # Of course, we need to remove nodes to check if a graph is critical
                    dictNode = coloring.get(node, None)
                    dictNeighbor = coloring.get(neighbor, None)
                    
                    if (dictNode != None and dictNeighbor != None):
                        if dictNode == dictNeighbor:
                            return 0
            
            max_color = max(coloring[node] for node in adjacency_dict.keys())

            # Return the maximum color
            return max_color

        # Otherwise, recursively try all possible color assignments for the next node
        next_node = node_order[len(coloring)]
        neighborhood = adjacency_dict[next_node]
        
        for color in range(max_degree + 1):
            if all(color != coloring.get(neighbor, None) for neighbor in neighborhood):
                coloring[next_node] = color
                
                max_color = try_coloring(coloring, node_order)
                
                if (max_color != 0):
                    # Return the maximum color
                    return max_color
        return 0

    # Generate a list of nodes ordered by degree (highest first)
    nodes = list(adjacency_dict.keys())
    node_order = sorted(nodes, key=lambda node: -len(adjacency_dict[node]))

    # Try all possible colorings starting with an empty coloring
    colors = list(range(max_degree + 1))
    chi = try_coloring({}, node_order)
    # criticalCache[graph6DictString] = chi
    testStrCache[STRdictString] = chi
    # print(cache[dictString])
    return chi + 1

def adjacencyDict(edges):
        # Build a dictionary where each edge is a key and its value is a set of neighboring edges
    adjacency_dict = {}
    for edge in edges:
        if edge[0] not in adjacency_dict:
            adjacency_dict[edge[0]] = set()
        if edge[1] not in adjacency_dict:
            adjacency_dict[edge[1]] = set()
        adjacency_dict[edge[0]].add(edge[1])
        adjacency_dict[edge[1]].add(edge[0])
    
    print(adjacency_dict)
    return adjacency_dict

def remove_node(adj_dict, node):
    # create a copy of the adjacency dictionary to modify
    new_adj_dict = adj_dict.copy()

    # remove the node and its edges from the dictionary
    
    #This takes care of the node
    del new_adj_dict[node]
    
    #This loops through the edges
    for neighbor in adj_dict[node]:
        
        getter = new_adj_dict.get(neighbor, None)
        if (getter != None) :
            if (node in new_adj_dict[neighbor]):
                new_adj_dict[neighbor].remove(node)
    # del new_adj_dict[node]

  # shift down the node indices higher than the removed node
    for i in range(node+1, max(new_adj_dict)+1):
        if i in new_adj_dict:
            new_adj_dict[i-1] = new_adj_dict.pop(i)
            new_adj_dict[i-1] = {j-1 if j>node else j for j in new_adj_dict[i-1]}
  

    return new_adj_dict

#This returns whether a vertex can be deleted while still keeping the chromatic number the same
def isCriticalOnK(adjacency_dict, k):
    
    #We can't have a k-critical graph when there's fewer than k nodes
    #However, we can have a critical graph with k or more nodes depending on where the edges are
    if (len(adjacency_dict) < k):
        return False
    
    #A k-critical graph has exactly k as its chromatic number
    chi = has_valid_coloring(adjacency_dict)
    if (chi != k):
        return False;
    
    
    #For any node that we try to remove, the graph is critical when it loses chromatic number
    for node in adjacency_dict:
        
        #Graphs with isolated vertices are never critical.
        #You can color an isolated vertex whatever you want
        #if (len(adjacency_dict[node]) == 0):
            #print(adjacency_dict)
            #return False
        
        temp_dict = remove_node(adjacency_dict, node)
        chiCrit = has_valid_coloring(temp_dict)
        
        #If the chromatic number stays the same for a node we remove, the graph is not critical
        #Every node must be necessary to keep chi where it is.
        #If there's nodes that aren't strictly necessary, we're hooped
        if (chiCrit == k):
            return False
    
    #The graph has passed all the tests
    return True

def graph_to_graph6_3(adj_dict):
    
    dictString = str(adj_dict)
    
    #This cache can get big, so we only do it for small graphs
    if (len(adj_dict) <= 5):
        getter = graph6Cache.get(dictString, None)
        if (getter != None):
                return getter
    
    # Create a graph object from the adjacency dictionary
    G = nx.Graph(adj_dict)

    # Obtain a Graph6 string from the graph object
    graph6 = nx.to_graph6_bytes(G).decode('ascii')

    # Remove the newline character at the end of the string
    graph6 = graph6.rstrip('\n')
    
    if (len(adj_dict) <= 5):
        graph6Cache[dictString] = graph6
    
    return graph6

def removeSubstring(myStr, substring):
    output_string = ""
    str_list = myStr.split(substring)
    for element in str_list:
        output_string += element
    return output_string

def subdirectory(subdirectoryName, number):
    # Get the current directory path
    current_directory = os.path.dirname(os.path.abspath(__file__))


    # Create the subdirectory if it doesn't exist
    subdirectory_path = os.path.join(current_directory, subdirectoryName)
    os.makedirs(subdirectory_path, exist_ok=True)

    # Create the file within the subdirectory
    file_path = os.path.join(subdirectory_path, "classified graphs " + str(number) + ".txt")
    return file_path

def showGraph(dict_):
    #print(dict_)
    
    G = nx.Graph(dict_)
                        
    color_array = ["red", "blue", "green", "gold", "orange", "purple", "teal", "black", "grey", "steelblue", "magenta", "violet", "dodgerblue", "brown"]
    color_array_trunc = []
    
    for i in range(len(dict_)):
        color_array_trunc.append(color_array[i])
        
    # Create a dictionary that maps each node to its color
    # color_map = {node: color_array[coloring[node]] for node in adjacency_dict.keys()}

    # Draw the colored graph using networkx
    nx.draw(G, with_labels=True, node_color=color_array_trunc, font_color='w')
    plt.show()

def classifyGraph(adjacencyDict, minK, maxK):
  
    #We're looking for 
    for i in range(minK, maxK + 1):
        
        if (isCriticalOnK(adjacencyDict, i)):
            # print("Critical graph found!")
            return [True, "The graph is critical on: " + str(i)]
    
    return [False, "The graph is not critical in the k range of " + str(minK) + " - " + str(maxK)]



def foo(inputArray):
        
    global imgDirectory
    global currentGraphFile
    global foundGraphs
    global outputLine
    global outputStream

    edges = inputArray[0]
    nodes = inputArray[1]
    
    adj_dict = {node: set() for node in nodes}
    for edge in edges:
        if (len(edge) == 2):
            adj_dict[edge[0]].add(edge[1])
            adj_dict[edge[1]].add(edge[0])


    # This array has True in index 0 if the graph is critical
    # It ha a print message in index 1
    graphResults = classifyGraph(adj_dict, min_k, max_k)
    if (graphResults[0]):

        graph6Str = graph_to_graph6_3(adj_dict)
        graph6Str = removeSubstring(graph6Str, ">>graph6<<")

        global writeToFile
        if (writeToFile):
            outputLine += graph6Str
            # outputLine = str(adj_dict)
            outputLine += "\n"
            #outputLine += graphResults[1]
            #outputLine += "\n"
            foundGraphs = foundGraphs + 1

            

        #print(edges)
        #print(graphResults[1])
        #showGraph(adj_dict)

    if (foundGraphs > 500):
        outputStream.write(outputLine)
        outputStream.close()
        outputLine = ""
        print("File " + str(currentGraphFile) + " Created!")
        currentGraphFile = currentGraphFile + 1

        path = os.path.join(imgDirectory, f"graph_{currentGraphFile}.g6")
        outputStream = open(path, "w")

        return
                    
def multithreading(min_nodes, max_nodes, min_k, max_k, num_threads):
    
    global imgDirectory
    global currentGraphFile
    global foundGraphs
    global outputStream
    
    if (not os.path.exists(imgDirectory)):
        os.makedirs(imgDirectory)
    
    # Create a thread pool executor with the specified number of threads

    with ThreadPoolExecutor(max_workers=num_threads) as executor:

        # Submit each number to the executor for processing

        futures = []
        

        path = os.path.join(imgDirectory, f"graph_{currentGraphFile}.g6")
        outputStream = open(path, "w")


        for num_nodes in range(min_nodes, max_nodes+1):
            nodes = list(range(0, num_nodes))
            for num_edges in range(num_nodes*(num_nodes-1)//2 + 1):

                edge_combos = combinations(combinations(nodes, 2), num_edges)

                for edges in edge_combos:
                    futures.append(executor.submit(foo,[edges, nodes]))


        # Get the results from each future as they become available

        results = [future.result() for future in futures]
        outputStream.close()
        # for r in results:
        #    foo2(r)
            
    return

                    
                    
min_nodes = int(input("Enter the min number of nodes: "))
max_nodes = int(input("Enter the max number of nodes: "))

min_k = int(input("Enter the min number of k: "))
max_k = int(input("Enter the max number of k: "))

threads = int(input("Enter the number of threads: "))


print("Starting!")

start_time = datetime.datetime.now()

multithreading(min_nodes, max_nodes, min_k, max_k, threads)

end_time = datetime.datetime.now()

elapsed_time = end_time - start_time
totalSeconds = elapsed_time.total_seconds()



hours, remainder = divmod(int(totalSeconds), 3600)
minutes, seconds = divmod(remainder, 60)
roundSeconds = float("{:.2f}".format(math.modf(totalSeconds)[0] + seconds))

print("Elapsed time: {} hours, {} minutes, {} seconds".format(int(hours), int(minutes), roundSeconds))

Enter the min number of nodes: 4
Enter the max number of nodes: 6
Enter the min number of k: 4
Enter the max number of k: 7
Enter the number of threads: 9
Starting!
File 0 Created!
File 1 Created!
File 2 Created!
File 3 Created!
File 4 Created!
File 5 Created!
File 6 Created!
File 7 Created!
File 8 Created!
File 9 Created!
File 10 Created!
File 11 Created!
File 12 Created!
File 13 Created!
File 14 Created!
File 15 Created!
File 16 Created!
File 17 Created!
File 18 Created!
File 19 Created!
File 20 Created!
File 21 Created!
File 22 Created!
File 23 Created!
File 24 Created!
File 25 Created!
File 26 Created!
File 27 Created!
File 28 Created!
File 29 Created!
File 30 Created!
File 31 Created!
File 32 Created!
File 33 Created!
File 34 Created!
File 35 Created!
File 36 Created!
File 37 Created!
File 38 Created!
File 39 Created!
File 40 Created!
File 41 Created!
File 42 Created!
File 43 Created!
File 44 Created!
File 45 Created!
File 46 Created!
File 47 Created!
File 48 Created!
File 49 Crea

ValueError: write to closed file